# Data Preprocessing

In [36]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import scipy.sparse
import joblib
from collections import Counter
import re
import string
import os
import pickle
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nont\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nont\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nont\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Parameters

In [37]:
# change this accordingly
project_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

save_path = f"{project_path}/scicite_preprocessed"

In [38]:
def load_result(file_path):
    with open(file_path,'rb') as f:
        payload = pickle.load(f)
        print("loaded!")
    return payload

In [39]:
selected_features = load_result(r'E:\cs4248_proj\CS4248-NLP-Project\feature_selection_result\multi-collinear-selected_featurename.pkl')

loaded!


## 1. Load dataset

In [40]:
dataset_dir = f"{project_path}/scicite"
train_df = pd.read_json(f"{dataset_dir}/train.jsonl", lines=True)
test_df = pd.read_json(f"{dataset_dir}/test.jsonl", lines=True)

display(
    train_df.head(),
    train_df.shape,
    test_df.head(),
    test_df.shape
)

,source,citeEnd,sectionName,citeStart,string,label,label_confidence,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label2_confidence
0,explicit,175.0,Introduction,168.0,"However, how frataxin interacts with the Fe-S ...",background,1.0,1872080baa7d30ec8fb87be9a65358cd3a7fb649,894be9b4ea46a5c422e81ef3c241072d4c73fdc0,True,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,1872080baa7d30ec8fb87be9a65358cd3a7fb649>894be...,11,NaN,NaN
1,explicit,36.0,Novel Quantitative Trait Loci for Seminal Root...,16.0,"In the study by Hickey et al. (2012), spikes w...",background,1.0,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b,b6642e19efb8db5623b3cc4eef1c5822a6151107,True,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,ce1d09a4a3a8d7fd3405b9328f65f00c952cf64b>b6642...,2,NaN,NaN
2,explicit,228.0,Introduction,225.0,"The drug also reduces catecholamine secretion,...",background,1.0,9cdf605beb1aa1078f235c4332b3024daa8b31dc,4e6a17fb8d7a3cada601d942e22eb5da6d01adbd,False,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,9cdf605beb1aa1078f235c4332b3024daa8b31dc>4e6a1...,0,NaN,NaN
3,explicit,110.0,Discussion,46.0,By clustering with lowly aggressive close kin ...,background,1.0,d9f3207db0c79a3b154f3875c9760cc6b056904b,2cc6ff899bf17666ad35893524a4d61624555ed7,False,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,d9f3207db0c79a3b154f3875c9760cc6b056904b>2cc6f...,3,NaN,NaN
4,explicit,239.0,Discussion,234.0,Ophthalmic symptoms are rare manifestations of...,background,1.0,88b86556857f4374842d2af2e359576806239175,a5bb0ff1a026944d2a47a155462959af2b8505a8,False,88b86556857f4374842d2af2e359576806239175>a5bb0...,88b86556857f4374842d2af2e359576806239175>a5bb0...,2,NaN,NaN


(8243, 15)

,source,citeEnd,sectionName,citeStart,string,label,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label_confidence
0,acronym,31.0,,28.0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,2c6797dab4c118cb73197f65ba39dacc99ac743d,95c37bc99982d33873fd141ee00857160fd717a0,True,2c6797dab4c118cb73197f65ba39dacc99ac743d>95c37...,2c6797dab4c118cb73197f65ba39dacc99ac743d>95c37...,6,NaN,NaN
1,explicit,166.0,Discussion,156.0,"In addition, the result of the present study s...",result,fa7145adc9f8cfb8af7a189d9040c13c84ced094,20e23b4f76761d246a7c3b00b80e139e2008f77d,False,fa7145adc9f8cfb8af7a189d9040c13c84ced094>20e23...,fa7145adc9f8cfb8af7a189d9040c13c84ced094>20e23...,0,supportive,NaN
2,explicit,145.0,Discussion,137.0,Several instruments that more specifically add...,background,98a8d8c0c5dae246720d4f339b88e8a9f44e3002,bd222c7ec83dadefba513738290b3624f6dd6b21,True,98a8d8c0c5dae246720d4f339b88e8a9f44e3002>bd222...,98a8d8c0c5dae246720d4f339b88e8a9f44e3002>bd222...,1,NaN,NaN
3,explicit,158.0,Methods,137.0,Organotypic hippocampal slice cultures\nInterf...,method,aeb178ef1910a61152cd74209c28641199c82855,754c04953c261072fa367f4104e3deff082d9484,False,aeb178ef1910a61152cd74209c28641199c82855>754c0...,aeb178ef1910a61152cd74209c28641199c82855>754c0...,1,NaN,1.0
4,explicit,186.0,RESULTS,178.0,Activated PBMC are the basis of the standard P...,background,e4d2591ac3bb65e2ec59f092884a7b15b8018592,f0fb468a54fe8021bc7986a1618222c4fcd16df4,False,e4d2591ac3bb65e2ec59f092884a7b15b8018592>f0fb4...,e4d2591ac3bb65e2ec59f092884a7b15b8018592>f0fb4...,6,NaN,NaN


(1861, 14)

## 2. Data Cleaning

##### Final Columns
* source: text -> categorical
* citeEnd: float
* citeStart: float
* sectionName: text -> categorical
* string: text -> embedding (vectorized)
* iskeyCitation: bool -> categorical
* excerpt index: int
* label: text -> categorical

#### 2.1 Drop unwanted columns

In [41]:
train_df = train_df.drop(columns=['label_confidence','citingPaperId','citedPaperId','id','unique_id','label2','label2_confidence'])
test_df = test_df.drop(columns=['label_confidence','citingPaperId','citedPaperId','id','unique_id','label2'])

display(
    train_df.columns,
    train_df.shape,
    test_df.columns,
    test_df.shape
)

Index(['source', 'citeEnd', 'sectionName', 'citeStart', 'string', 'label',
       'isKeyCitation', 'excerpt_index'],
      dtype='object')

(8243, 8)

Index(['source', 'citeEnd', 'sectionName', 'citeStart', 'string', 'label',
       'isKeyCitation', 'excerpt_index'],
      dtype='object')

(1861, 8)

#### 2.2 Drop selected null rows

In [42]:
# Define a function to check null values
def check_nulls(df, name):
    print(f"\n{name} dataset:")
    for column in df.columns:
        print(f"{column} null value:", df[column].isnull().sum())

# Check null values for both train and test datasets
check_nulls(train_df, "Train")
check_nulls(test_df, "Test")


Train dataset:
source null value: 2
citeEnd null value: 2
sectionName null value: 19
citeStart null value: 2
string null value: 0
label null value: 0
isKeyCitation null value: 0
excerpt_index null value: 0

Test dataset:
source null value: 3
citeEnd null value: 3
sectionName null value: 10
citeStart null value: 3
string null value: 0
label null value: 0
isKeyCitation null value: 0
excerpt_index null value: 0


In [43]:
def drop_and_display_nulls(df, name):
    df = df.dropna(subset=['source', 'citeEnd', 'citeStart'])
    print(f"\n{name} dataset:")
    for column in ['source', 'citeEnd', 'citeStart']:
        print(f"Number of null values in '{column}': {df[column].isnull().sum()}")
    return df

# Apply function to both train and test datasets
train_df = drop_and_display_nulls(train_df, "Train")
test_df = drop_and_display_nulls(test_df, "Test")


Train dataset:
Number of null values in 'source': 0
Number of null values in 'citeEnd': 0
Number of null values in 'citeStart': 0

Test dataset:
Number of null values in 'source': 0
Number of null values in 'citeEnd': 0
Number of null values in 'citeStart': 0


#### 2.3 One-hot encoding for 'source' and 'isKeyCitation' columns

In [44]:
def one_hot_encode_and_concat(df, column_name):
    one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    
    # Fit and transform the specified column
    one_hot_encoded = one_hot_encoder.fit_transform(df[[column_name]])
    one_hot_columns = one_hot_encoder.get_feature_names_out([column_name])
    
    # Create DataFrame for one-hot encoded values
    df_one_hot = pd.DataFrame(one_hot_encoded, columns=one_hot_columns, index=df.index)
    
    # Concatenate the one-hot encoded DataFrame with the original DataFrame
    df = pd.concat([df, df_one_hot], axis=1)
    
    # Drop the original categorical column
    df.drop(columns=[column_name], inplace=True)
    
    return df, one_hot_columns

# Apply the function to both train and test datasets
train_df, source_columns_train = one_hot_encode_and_concat(train_df, 'source')
train_df, key_citation_columns_train = one_hot_encode_and_concat(train_df, 'isKeyCitation')

test_df, source_columns_test = one_hot_encode_and_concat(test_df, 'source')
test_df, key_citation_columns_test = one_hot_encode_and_concat(test_df, 'isKeyCitation')

# Show the mappings for both datasets
print("Train Source Mapping:", source_columns_train)
print("Train Key Citation Mapping:", key_citation_columns_train)
print("Train DataFrame Shape:", train_df.shape)

print("\nTest Source Mapping:", source_columns_test)
print("Test Key Citation Mapping:", key_citation_columns_test)
print("Test DataFrame Shape:", test_df.shape)

Train Source Mapping: ['source_acronym' 'source_acronymParen' 'source_andPhrase'
 'source_etAlPhrase' 'source_explicit' 'source_properNoun']
Train Key Citation Mapping: ['isKeyCitation_False' 'isKeyCitation_True']
Train DataFrame Shape: (8241, 14)

Test Source Mapping: ['source_acronym' 'source_acronymParen' 'source_andPhrase'
 'source_etAlPhrase' 'source_explicit' 'source_properNoun']
Test Key Citation Mapping: ['isKeyCitation_False' 'isKeyCitation_True']
Test DataFrame Shape: (1858, 14)


#### 2.4 Clean 'sectionName' column

- replace NaN and empty strings with "Unknown"

In [45]:
def replace_empty_and_nan(df, column_name):
    # Before replacement
    num_nan = df[column_name].isna().sum()
    num_empty = (df[column_name] == '').sum()
    print(f"Before replacement in '{column_name}':")
    print(f"NaN values: {num_nan}")
    print(f"Empty strings: {num_empty}")

    # Replacement
    df[column_name] = df[column_name].replace('', np.nan).fillna("Unknown")

    # After replacement
    num_nan = df[column_name].isna().sum()
    num_empty = (df[column_name] == '').sum()
    print(f"\nAfter replacement in '{column_name}':")
    print(f"NaN values: {num_nan}")
    print(f"Empty strings: {num_empty}")

# Apply to both train_df and test_df
print("Train dataset:")
replace_empty_and_nan(train_df, 'sectionName')

print("\nTest dataset:")
replace_empty_and_nan(test_df, 'sectionName')

Train dataset:
Before replacement in 'sectionName':
NaN values: 19
Empty strings: 587

After replacement in 'sectionName':
NaN values: 0
Empty strings: 0

Test dataset:
Before replacement in 'sectionName':
NaN values: 10
Empty strings: 66

After replacement in 'sectionName':
NaN values: 0
Empty strings: 0


- convert string to lowercase

In [46]:
def convert_to_lowercase(df, column_name):
    # Before conversion
    num_unique_before = len(df[column_name].unique())
    print(f"Num unique strings before lowercase in '{column_name}': {num_unique_before}")

    # Conversion to lowercase
    df[column_name] = df[column_name].apply(str.lower)

    # After conversion
    num_unique_after = len(df[column_name].unique())
    print(f"Num unique strings after lowercase in '{column_name}': {num_unique_after}")

# Apply to both train_df and test_df
print("Train dataset:")
convert_to_lowercase(train_df, 'sectionName')

print("\nTest dataset:")
convert_to_lowercase(test_df, 'sectionName')


Train dataset:
Num unique strings before lowercase in 'sectionName': 1147
Num unique strings after lowercase in 'sectionName': 1074

Test dataset:
Num unique strings before lowercase in 'sectionName': 301
Num unique strings after lowercase in 'sectionName': 271


- remove numbers and punctuation

In [47]:
def remove_punctuation_and_numbers(df, column_name):
    # Before removal
    num_unique_before = len(df[column_name].unique())
    print(f"Num unique strings before removing in '{column_name}': {num_unique_before}")

    # Remove numbers and punctuation
    df[column_name] = df[column_name].str.replace(r'[\d{}]+'.format(string.punctuation), '', regex=True).str.strip()

    # After removal
    num_unique_after = len(df[column_name].unique())
    print(f"Num unique strings after removing in '{column_name}': {num_unique_after}")

# Apply to both train_df and test_df
print("Train dataset:")
remove_punctuation_and_numbers(train_df, 'sectionName')

print("\nTest dataset:")
remove_punctuation_and_numbers(test_df, 'sectionName')

Train dataset:
Num unique strings before removing in 'sectionName': 1074
Num unique strings after removing in 'sectionName': 807

Test dataset:
Num unique strings before removing in 'sectionName': 271
Num unique strings after removing in 'sectionName': 162


- lemmatization

In [48]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = nltk.word_tokenize(text)  # Tokenize the text into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    return " ".join(lemmatized_words)

def lemmatize_column(df, column_name):
    # Before lemmatization
    num_unique_before = len(df[column_name].unique())
    print(f"Num unique strings before lemmatization in '{column_name}': {num_unique_before}")

    # Lemmatize the text in the column
    df[column_name] = df[column_name].apply(lemmatize_text)

    # After lemmatization
    num_unique_after = len(df[column_name].unique())
    print(f"Num unique strings after lemmatization in '{column_name}': {num_unique_after}")

# Apply to both train_df and test_df
print("Train dataset:")
lemmatize_column(train_df, 'sectionName')

print("\nTest dataset:")
lemmatize_column(test_df, 'sectionName')

Train dataset:
Num unique strings before lemmatization in 'sectionName': 807
Num unique strings after lemmatization in 'sectionName': 794

Test dataset:
Num unique strings before lemmatization in 'sectionName': 162
Num unique strings after lemmatization in 'sectionName': 158


- mapping to pre-defined categories

In [49]:
# Keywords to be mapped to 'background'
background_keywords = ['background', 'introduction', 'related work', 'relatedwork', 'overview', 
            'literature review', 'summary', 'state of the art', 'prior work']

pattern = re.compile('|'.join(background_keywords), re.IGNORECASE)

def map_background(df, column_name):
    # Before mapping
    num_unique_before = len(df[column_name].unique())
    print(f"Num unique strings before mapping in '{column_name}': {num_unique_before}")

    # Apply the mapping
    df[column_name] = df[column_name].apply(lambda text: 'background' if pattern.search(text) else text)

    # After mapping
    num_unique_after = len(df[column_name].unique())
    print(f"Num unique strings after mapping in '{column_name}': {num_unique_after}")

# Apply to both train_df and test_df
print("Train dataset:")
map_background(train_df, 'sectionName')

print("\nTest dataset:")
map_background(test_df, 'sectionName')


Train dataset:
Num unique strings before mapping in 'sectionName': 794
Num unique strings after mapping in 'sectionName': 765

Test dataset:
Num unique strings before mapping in 'sectionName': 158
Num unique strings after mapping in 'sectionName': 153


In [50]:
# Keywords to be mapped to 'method'
method_keywords = ['methodology', 'method', 'approach', 'design', 
                   'proposed', 'experiment', 'implementation', 'procedure', 
                   'description', 'technique', 'technical', 'formulation', 
                   'solution', 'collection', 'describ', 'measure', 'model', 
                   'empirical', 'study', 'setup', 'setting', 'system', 'detail']

pattern = re.compile('|'.join(method_keywords), re.IGNORECASE)

def map_method(df, column_name):
    # Before mapping
    num_unique_before = len(df[column_name].unique())
    print(f"Num unique strings before mapping in '{column_name}': {num_unique_before}")

    # Apply the mapping
    df[column_name] = df[column_name].apply(lambda text: 'method' if pattern.search(text) else text)

    # After mapping
    num_unique_after = len(df[column_name].unique())
    print(f"Num unique strings after mapping in '{column_name}': {num_unique_after}")

# Apply to both train_df and test_df
print("Train dataset:")
map_method(train_df, 'sectionName')

print("\nTest dataset:")
map_method(test_df, 'sectionName')


Train dataset:
Num unique strings before mapping in 'sectionName': 765
Num unique strings after mapping in 'sectionName': 625

Test dataset:
Num unique strings before mapping in 'sectionName': 153
Num unique strings after mapping in 'sectionName': 125


In [51]:
# Keywords to be mapped to 'result'
result_keywords = ['result', 'discussion', 'evaluation', 'limitation', 'implication', 'comparison', 
                   'contribution', 'analysis', 'test', 'compare']

pattern = re.compile('|'.join(result_keywords), re.IGNORECASE)

def map_result(df, column_name):
    # Before mapping
    num_unique_before = len(df[column_name].unique())
    print(f"Num unique strings before mapping in '{column_name}': {num_unique_before}")

    # Apply the mapping
    df[column_name] = df[column_name].apply(lambda text: 'result' if pattern.search(text) else text)

    # After mapping
    num_unique_after = len(df[column_name].unique())
    print(f"Num unique strings after mapping in '{column_name}': {num_unique_after}")

# Apply to both train_df and test_df
print("Train dataset:")
map_result(train_df, 'sectionName')

print("\nTest dataset:")
map_result(test_df, 'sectionName')


Train dataset:
Num unique strings before mapping in 'sectionName': 625
Num unique strings after mapping in 'sectionName': 557

Test dataset:
Num unique strings before mapping in 'sectionName': 125
Num unique strings after mapping in 'sectionName': 106


In [52]:
# Keywords to be mapped to 'conclusion'
conclusion_keywords = ['conclusion', 'concluding', 'future']

pattern = re.compile('|'.join(conclusion_keywords), re.IGNORECASE)

def map_conclusion(df, column_name):
    # Before mapping
    num_unique_before = len(df[column_name].unique())
    print(f"Num unique strings before mapping in '{column_name}': {num_unique_before}")

    # Apply the mapping
    df[column_name] = df[column_name].apply(lambda text: 'conclusion' if pattern.search(text) else text)

    # After mapping
    num_unique_after = len(df[column_name].unique())
    print(f"Num unique strings after mapping in '{column_name}': {num_unique_after}")

# Apply to both train_df and test_df
print("Train dataset:")
map_conclusion(train_df, 'sectionName')

print("\nTest dataset:")
map_conclusion(test_df, 'sectionName')


Train dataset:
Num unique strings before mapping in 'sectionName': 557
Num unique strings after mapping in 'sectionName': 551

Test dataset:
Num unique strings before mapping in 'sectionName': 106
Num unique strings after mapping in 'sectionName': 106


In [53]:
# List of valid section names
valid_sections = {'background', 'result', 'conclusion', 'method', 'unknown'}

def map_to_valid_sections(df, column_name, valid_sections):
    # Before mapping
    num_unique_before = len(df[column_name].unique())
    print(f"Num unique strings before mapping in '{column_name}': {num_unique_before}")

    # Apply the mapping
    df[column_name] = df[column_name].apply(lambda text: text if text in valid_sections else 'unknown')

    # After mapping
    num_unique_after = len(df[column_name].unique())
    print(f"Num unique strings after mapping in '{column_name}': {num_unique_after}")

# Apply to both train_df and test_df
print("Train dataset:")
map_to_valid_sections(train_df, 'sectionName', valid_sections)

print("\nTest dataset:")
map_to_valid_sections(test_df, 'sectionName', valid_sections)


Train dataset:
Num unique strings before mapping in 'sectionName': 551
Num unique strings after mapping in 'sectionName': 5

Test dataset:
Num unique strings before mapping in 'sectionName': 106
Num unique strings after mapping in 'sectionName': 4


In [54]:
def apply_one_hot_encoding(df, column_name, one_hot_encoder=None):
    if one_hot_encoder is None:
        # Initialize OneHotEncoder
        one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        # Fit the encoder on the training dataset
        one_hot_encoder = one_hot_encoder.fit(df[[column_name]])
    # Transform both train_df and test_df using the fitted encoder
    encoded = one_hot_encoder.transform(df[[column_name]])
    
    # Get one-hot encoded column names
    one_hot_columns = one_hot_encoder.get_feature_names_out([column_name])
    
    # Convert the result to a DataFrame
    one_hot_df = pd.DataFrame(encoded, columns=one_hot_columns, index=df.index)
    
    # Concatenate the original DataFrame with the one-hot encoded columns
    df = pd.concat([df, one_hot_df], axis=1)
    
    # Drop the original column
    df.drop(columns=[column_name], inplace=True)
    
    return df, one_hot_encoder

# Apply to train_df and test_df
print("Train dataset:")
train_df, one_hot_encoder = apply_one_hot_encoding(train_df, 'sectionName', one_hot_encoder=None)

print("\nTest dataset:")
test_df, _ = apply_one_hot_encoding(test_df, 'sectionName', one_hot_encoder)

# Display the results
display(
    train_df.head(),
    test_df.head()
)

Train dataset:

Test dataset:


,citeEnd,citeStart,string,label,excerpt_index,source_acronym,source_acronymParen,source_andPhrase,source_etAlPhrase,source_explicit,source_properNoun,isKeyCitation_False,isKeyCitation_True,sectionName_background,sectionName_conclusion,sectionName_method,sectionName_result,sectionName_unknown
0,175.0,168.0,"However, how frataxin interacts with the Fe-S ...",background,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,36.0,16.0,"In the study by Hickey et al. (2012), spikes w...",background,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,228.0,225.0,"The drug also reduces catecholamine secretion,...",background,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,110.0,46.0,By clustering with lowly aggressive close kin ...,background,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,239.0,234.0,Ophthalmic symptoms are rare manifestations of...,background,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


,citeEnd,citeStart,string,label,excerpt_index,source_acronym,source_acronymParen,source_andPhrase,source_etAlPhrase,source_explicit,source_properNoun,isKeyCitation_False,isKeyCitation_True,sectionName_background,sectionName_conclusion,sectionName_method,sectionName_result,sectionName_unknown
0,31.0,28.0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,166.0,156.0,"In addition, the result of the present study s...",result,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,145.0,137.0,Several instruments that more specifically add...,background,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,158.0,137.0,Organotypic hippocampal slice cultures\nInterf...,method,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,186.0,178.0,Activated PBMC are the basis of the standard P...,background,6,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


#### 2.5 Clean 'string' column
- Remove citations (e.g., [12,22]).
- Convert text to lowercase.
- Remove punctuation.
- Remove numbers.
- Remove stopwords.
- Lemmatize words using NLTK.
- Replace multiple spaces with a single space.

In [55]:
# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to clean text
def clean_text(text):
    if pd.isna(text):  # Handle NaN values
        return ""
    
    # Remove various citation formats
    text = re.sub(r"\[[\d,\s–-]+\]", "", text)  # Remove citations like [1–5], [21, 23, 33, 64, 100, 101, 107–118]
    text = re.sub(r"\([\d,\s–-]+\)", "", text)  # Remove citations like (96-98), (7)
    text = re.sub(r"\b[A-Z][a-z]+ et al\.\s*\(\d{4}\)", "", text)  # Doe et al. (2021)
    text = re.sub(r"\(\w+,\s*\d{4}(?:;\s*\w+,\s*\d{4})*\)", "", text)  # (Smith, 2020; Johnson, 2019)
    text = re.sub(r"\b(ICML|NeurIPS|CVPR|JMLR|arXiv|vol\.\s*\d+)\b", "", text)  # Journal/conference names

    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\d+", "", text)  # Remove numbers
    
    tokens = word_tokenize(text)  # Tokenize text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Lemmatization & remove stopwords
    
    return " ".join(tokens)  # Reconstruct sentence

# Apply text cleaning function
train_df['string'] = train_df['string'].apply(clean_text)
test_df['string'] = test_df['string'].apply(clean_text)

# Display results
display(
    train_df.head(),
    test_df.head()
)

,citeEnd,citeStart,string,label,excerpt_index,source_acronym,source_acronymParen,source_andPhrase,source_etAlPhrase,source_explicit,source_properNoun,isKeyCitation_False,isKeyCitation_True,sectionName_background,sectionName_conclusion,sectionName_method,sectionName_result,sectionName_unknown
0,175.0,168.0,however frataxin interacts fe cluster biosynth...,background,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,36.0,16.0,study hickey et al spike sampled field point p...,background,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,228.0,225.0,drug also reduces catecholamine secretion ther...,background,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,110.0,46.0,clustering lowly aggressive close kin king ab ...,background,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,239.0,234.0,ophthalmic symptom rare manifestation intracra...,background,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


,citeEnd,citeStart,string,label,excerpt_index,source_acronym,source_acronymParen,source_andPhrase,source_etAlPhrase,source_explicit,source_properNoun,isKeyCitation_False,isKeyCitation_True,sectionName_background,sectionName_conclusion,sectionName_method,sectionName_result,sectionName_unknown
0,31.0,28.0,chapel well x upc coarray fortran titanium rel...,background,6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,166.0,156.0,addition result present study support previous...,result,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,145.0,137.0,several instrument specifically address patien...,background,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,158.0,137.0,organotypic hippocampal slice culture interfac...,method,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,186.0,178.0,activated pbmc basis standard pbmc blast assay...,background,6,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


- TF-IDF Vectorization of 'string' column

In [56]:
# Initialize the vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)

# Fit and transform the 'string' column in the training set
X_train_tfidf = vectorizer.fit_transform(train_df["string"])

# Transform the 'string' column in the test set
X_test_tfidf = vectorizer.transform(test_df["string"])

# Convert the sparse matrix to a DataFrame for training set
tfidf_train_df = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out(), index=train_df.index)

# Rename columns of the TF-IDF DataFrame to avoid overlap (add '_tfidf' suffix)
tfidf_train_df.columns = [col + '_tfidf' for col in tfidf_train_df.columns]

# Drop the original 'string' column from the train DataFrame
train_df.drop(columns=['string'], inplace=True)

# Concatenate the original DataFrame with the TF-IDF DataFrame
train_df = pd.concat([train_df, tfidf_train_df], axis=1)

# Repeat the process for the test DataFrame
tfidf_test_df = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out(), index=test_df.index)
tfidf_test_df.columns = [col + '_tfidf' for col in tfidf_test_df.columns]

# Drop the original 'string' column from the test DataFrame
test_df.drop(columns=['string'], inplace=True)

# Concatenate the original test DataFrame with the TF-IDF DataFrame
test_df = pd.concat([test_df, tfidf_test_df], axis=1)

# Display the updated train and test DataFrames
display(
    train_df.head(),
    test_df.head()
)

,citeEnd,citeStart,label,excerpt_index,source_acronym,source_acronymParen,source_andPhrase,source_etAlPhrase,source_explicit,source_properNoun,...,zheng et al_tfidf,zhou_tfidf,zhou et_tfidf,zhou et al_tfidf,zhu_tfidf,zinc_tfidf,zn_tfidf,zone_tfidf,äì_tfidf,μm_tfidf
0,175.0,168.0,background,11,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,36.0,16.0,background,2,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,228.0,225.0,background,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,110.0,46.0,background,3,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,239.0,234.0,background,2,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,citeEnd,citeStart,label,excerpt_index,source_acronym,source_acronymParen,source_andPhrase,source_etAlPhrase,source_explicit,source_properNoun,...,zheng et al_tfidf,zhou_tfidf,zhou et_tfidf,zhou et al_tfidf,zhu_tfidf,zinc_tfidf,zn_tfidf,zone_tfidf,äì_tfidf,μm_tfidf
0,31.0,28.0,background,6,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,166.0,156.0,result,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,145.0,137.0,background,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,158.0,137.0,method,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,186.0,178.0,background,6,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 2.6 Label encoding for 'label' column

In [57]:
# Initialize Label Encoder
label_encoder = LabelEncoder()

# Encode labels directly in the 'label' column
train_df["label"] = label_encoder.fit_transform(train_df["label"])
test_df["label"] = label_encoder.fit_transform(test_df["label"])
joblib.dump(label_encoder, f"{save_path}/label_encoder.pkl")

# Show label mappings
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Labels:", label_mapping)

Labels: {'background': np.int64(0), 'method': np.int64(1), 'result': np.int64(2)}


In [58]:
selected_features = selected_features.to_list()
selected_features.append("label")
train_df = train_df[selected_features]
test_df = test_df[selected_features]

## Save processed dataset

In [59]:
train_df.to_csv(f"{save_path}/train-multi-colinear-selected-features.csv", index=False)
test_df.to_csv(f"{save_path}/test-multi-colinear-selected-features.csv", index=False)

# TODO: ML pipeline

- [x] Clean data
- [ ] Upsampling
- [x] Word vectorization
- [ ] Word embedding
- [ ] PCA 
- [ ] top-k features selection
- [ ] Training the classfication model